# Advanced Usage
Using AnnSQL, we will demonstrate extended functionality and how to perform the following operations:

- Determine total counts per cell library
- Calculate total gene counts
- Normalize the counts to 10,000 reads per cell.
- Log transform the counts.
- Calculate the top 2000 highly variable genes.

###  Install the AnnSQL package

```bash
pip install annsql
```

### Import libraries, build, then open the database

In [1]:
from AnnSQL import AnnSQL
from MakeDb import MakeDb
import scanpy as sc
import os
adata = sc.datasets.pbmc3k()
adata.var_names_make_unique()
if os.path.exists("db/pbmc3k.asql"):
	os.remove("db/pbmc3k.asql")
MakeDb(adata=adata, db_name="pbmc3k", db_path="db/")
adata_sql = AnnSQL(db="db/pbmc3k.asql")

Time to make var_names unique:  18.767115592956543
Time to create X table schema:  0.24080705642700195
Time to insert X data:  9.258856296539307


### Calculate total counts per cell

In [2]:
adata_sql.calculate_total_counts(chunk_size=950) #if system memory is low, lower chunks to <=200 

Total Counts Calculation Started
Total Counts Calculation Complete


### View the total counts

In [3]:
adata_sql.query("SELECT total_counts FROM X ORDER BY total_counts DESC LIMIT 5 ")

,total_counts
0,15844.0
1,15301.0
2,10783.0
3,10359.0
4,10282.0


### Calculate counts per gene

In [4]:
adata_sql.calculate_gene_counts(chunk_size=950)

Updating Var Table
Update Query Error: Binder Error: Table "var" does not have a column with name "gene_counts"
Gene Counts Calculation Started
Gene Counts Calculation Complete


### View the gene counts

In [5]:
adata_sql.query("SELECT * FROM var ORDER BY gene_counts DESC LIMIT 5 ")

,gene_ids,gene_names_orig,gene_names,gene_counts
0,ENSG00000251562,MALAT1,MALAT1,161685.0
1,ENSG00000205542,TMSB4X,TMSB4X,124210.0
2,ENSG00000166710,B2M,B2M,121363.0
3,ENSG00000147403,RPL10,RPL10,88517.0
4,ENSG00000167526,RPL13,RPL13,77111.0


### Normalize cell expression counts to 10,000	

In [6]:
#lower chunk_size if system memory is low. Max supported chunk_size is 950 (DuckDB limitation)
adata_sql.expression_normalize(chunk_size=950) 

Expression Normalization Started
Expression Normalization Complete


### View a normalized gene

In [7]:
adata_sql.query("SELECT RER1 FROM X ORDER BY RER1 DESC LIMIT 5 ")

,RER1
0,219.324249
1,204.081635
2,35.587189
3,18.939394
4,17.401392


### Log transform the expression values

In [8]:
adata_sql.expression_log(log_type="LOG2", chunk_size=950) #LOG2 or LOG10

Log Transform Started
Log Transform Complete


### Examine a log transformed value

In [9]:
adata_sql.query("SELECT RER1 FROM X ORDER BY RER1 DESC LIMIT 5 ")

,RER1
0,7.776922
1,7.673003
2,5.153286
3,4.243319
4,4.121131


### Calculate Highly Variable Genes

In [10]:
adata_sql.calculate_variable_genes(chunk_size=950) 

Updating Var Table
Update Query Error: Binder Error: Table "var" does not have a column with name "variance"
Variance Calculation Complete


### View the top 50 highly variable genes

In [11]:
adata_sql.query("SELECT * FROM var ORDER BY variance DESC LIMIT 50 ")

,gene_ids,gene_names_orig,gene_names,gene_counts,variance
0,ENSG00000204287,HLA-DRA,HLA_DRA,16467.0,111.171577
1,ENSG00000011600,TYROBP,TYROBP,9381.0,107.384903
2,ENSG00000101439,CST3,CST3,11857.0,107.361053
3,ENSG00000090382,LYZ,LYZ,27666.0,107.279701
4,ENSG00000100097,LGALS1,LGALS1,8055.0,106.759933
5,ENSG00000163220,S100A9,S100A9,16326.0,105.228394
6,ENSG00000223865,HLA-DPB1,HLA_DPB1,7817.0,104.762688
7,ENSG00000196126,HLA-DRB1,HLA_DRB1,6829.0,104.746109
8,ENSG00000231389,HLA-DPA1,HLA_DPA1,6980.0,104.061783
9,ENSG00000008517,IL32,IL32,5708.0,102.100822


In [22]:
#perform the same using the adata object
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, n_top_genes=50, subset=True)
adata.var

,gene_ids,gene_names_orig,gene_names,highly_variable,means,dispersions,dispersions_norm
index,,,,,,,
LRRIQ3,ENSG00000162620,LRRIQ3,LRRIQ3,True,3.213468,9.071757,-0.243689
C2CD4D,ENSG00000225556,C2CD4D,C2CD4D,True,2.543835,8.961646,1.000000
UBE2Q1,ENSG00000160714,UBE2Q1,UBE2Q1,True,5.034218,9.036189,1.347446
KIF3C,ENSG00000084731,KIF3C,KIF3C,True,2.837004,8.877491,-0.152070
FAM98A,ENSG00000119812,FAM98A,FAM98A,True,4.404673,8.917180,-1.079913
MTIF2,ENSG00000085760,MTIF2,MTIF2,True,5.158188,8.983418,0.010936
TTN-AS1,ENSG00000237298,TTN-AS1,TTN_AS1,True,4.378264,9.023237,1.749578
HEMK1,ENSG00000114735,HEMK1,HEMK1,True,4.822398,9.000399,0.707107
LINC00886,ENSG00000240875,LINC00886,LINC00886,True,2.971364,8.985250,1.067325


In [30]:
method1 = adata_sql.query("SELECT gene_names_orig FROM var ORDER BY variance DESC LIMIT 50").values.flatten()
method2 = adata.var.gene_names_orig.values

#find which genes are common between the two methods
common_genes = set(method1).intersection(set(method2))

array(['LRRIQ3', 'C2CD4D', 'UBE2Q1', 'KIF3C', 'FAM98A', 'MTIF2',
       'TTN-AS1', 'HEMK1', 'LINC00886', 'IDUA', 'ARHGAP24', 'USP38',
       'MOCS2', 'CTB-113I20.2', 'RNF14', 'DOK3', 'HIST1H1B', 'PPT2-EGFL8',
       'ABCC10', 'UBE2D4', 'GJC3', 'CTB-152G17.6', 'TMEM140', 'KIAA0196',
       'CDKN2A', 'ABHD17B', 'GBGT1', 'LZTS2', 'KCNQ1OT1', 'TAF10',
       'MICALCL', 'MADD', 'PGM2L1', 'MTRF1', 'RAD51B', 'CEP128', 'TTC8',
       'DPH6', 'MGA', 'TADA2A', 'YPEL2', 'PXMP4', 'FAM210B', 'ZNF561',
       'SLC27A1', 'LINC00662', 'EID2', 'NAPA-AS1', 'ARVCF', 'BACE2'],
      dtype=object)